In [18]:
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset_builder
from datasets import load_dataset
import numpy as np
import evaluate
import torch

In [19]:
tokenizer = AutoTokenizer.from_pretrained("KETI-AIR-Downstream/long-ke-t5-base-translation-aihub-ko2en",model_max_length=128)
model = AutoModelForSeq2SeqLM.from_pretrained("KETI-AIR-Downstream/long-ke-t5-base-translation-aihub-ko2en")

loading file spiece.model from cache at C:\Users\mtx/.cache\huggingface\hub\models--KETI-AIR-Downstream--long-ke-t5-base-translation-aihub-ko2en\snapshots\ec267d5ac800975de819e962a1fb2cb0897f277a\spiece.model
loading file tokenizer.json from cache at C:\Users\mtx/.cache\huggingface\hub\models--KETI-AIR-Downstream--long-ke-t5-base-translation-aihub-ko2en\snapshots\ec267d5ac800975de819e962a1fb2cb0897f277a\tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at C:\Users\mtx/.cache\huggingface\hub\models--KETI-AIR-Downstream--long-ke-t5-base-translation-aihub-ko2en\snapshots\ec267d5ac800975de819e962a1fb2cb0897f277a\special_tokens_map.json
loading file tokenizer_config.json from cache at C:\Users\mtx/.cache\huggingface\hub\models--KETI-AIR-Downstream--long-ke-t5-base-translation-aihub-ko2en\snapshots\ec267d5ac800975de819e962a1fb2cb0897f277a\tokenizer_config.json
loading configuration file config.json from cache at C:\Users\mtx/.cac

In [20]:
def prepare_dataset(data):
    source_language = [value['ko'] for key, value in data.items()]
    target_language = [value['en'] for key, value in data.items()]
    return source_language, target_language

In [ ]:
train  = load_dataset("Moo/korean-parallel-corpora", split="train")
#test = load_dataset("Moo/korean-parallel-corpora", split="test")
#validation = load_dataset("Moo/korean-parallel-corpora", split="validation")

In [ ]:
train[0]

In [ ]:
inputs = tokenizer(train['ko'],return_tensors="pt", max_length=128, truncation=True,padding=True)
outputs = tokenizer(train['en'],return_tensors="pt",max_length=128, truncation=True,padding=True)

In [ ]:
dataset = torch.utils.data.TensorDataset(inputs.input_ids, inputs.attention_mask, outputs.input_ids, outputs.attention_mask)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    fp16=True,
    remove_unused_columns=False,
    logging_dir="./logs",
)

def data_collator(batch):
        return {
            "input_ids": torch.stack([item[0] for item in batch]),
            "attention_mask": torch.stack([item[1] for item in batch]),
            "labels": torch.stack([item[2] for item in batch]),
        }

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


import transformers
transformers.logging.set_verbosity_info()

trainer.train()